In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np
import random
import copy
import ast
from datetime import datetime
from collections import Counter, defaultdict


# Visualization packages
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Append system path
sys.path = [p for p in sys.path if not p.endswith('../..')]  # Cleans duplicated '../..'
sys.path.insert(0, '../')  # This adds `src` to the path

from helpers import io, filters, constants
from analysis import analysis_util, analysis_constants, visualization_util
from web_analysis import parse_robots
from web_analysis import robots_util


%load_ext autoreload
%autoreload 2

In [2]:
FPATH_TO_RELEVANT_URL_TOKENS = 'pretrain_data/relevant_url_token_counts.csv'
FPATH_TO_ALL_URL_TOKENS = 'pretrain_data/all_url_token_counts.csv'
FPATH_to_HEAD_ROBOTS = "robots_data/temporal_robots_head.json"
FPATH_TO_RAND_ROBOTS = "robots_data/temporal_robots_rand_10k.json"
FPATH_TO_TOS_DATA = "robots_data/tos_ai_scraping_policies.json"
DIRPATHS_TO_ANNOTATED_TASKS = ["annotated_websites/Task 1", "annotated_websites/Task 2"]
FPATH_SNAPSHOT_DATA = "robots_data/temporal_main_sites_current.json"

COMPANIES_TO_ANALYZE = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta"]
TEMPORAL_ANALYSIS_START_DATE = '2016-01-01'
TEMPORAL_ANALYSIS_END_DATE = '2024-04-30'

In [3]:
agent_groups_to_track = robots_util.get_bot_groups(COMPANIES_TO_ANALYZE)
agents_to_track = robots_util.get_bots()

In [4]:
url_token_lookup = robots_util.URLTokenLookup(FPATH_TO_RELEVANT_URL_TOKENS) # 'c4', 'rf', 'dolma'
c4_url_to_counts = url_token_lookup.get_url_to_token_map("c4")
rf_url_to_counts = url_token_lookup.get_url_to_token_map("rf")
dolma_url_to_counts = url_token_lookup.get_url_to_token_map("dolma")
top_c4_urls = url_token_lookup.top_k_urls("c4", 2000)
top_rf_urls = url_token_lookup.top_k_urls("rf", 2000)
top_dolma_urls = url_token_lookup.top_k_urls("dolma", 2000)
random_10k_urls = url_token_lookup.get_10k_random_sample()
all_urls = set(random_10k_urls + top_c4_urls + top_rf_urls + top_dolma_urls)

# Load website snapshots for relevant URLs
website_snapshots = robots_util.read_snapshots(FPATH_SNAPSHOT_DATA, all_urls)

Number of tokens in 2000 URLs: 18447797380 | 10.85% of c4
Number of tokens in 2000 URLs: 67098747294 | 15.56% of rf
Number of tokens in 2000 URLs: 429152555144 | 21.74% of dolma


In [5]:
# URL -> Date -> Robots.txt raw text
head_robots = io.read_json(FPATH_to_HEAD_ROBOTS)
random_10k_robots = io.read_json(FPATH_TO_RAND_ROBOTS)
joined_robots = copy.deepcopy(head_robots)
joined_robots.update(random_10k_robots)
robots_util.print_out_robots_info(head_robots)
robots_util.print_out_robots_info(random_10k_robots)

# {URL --> Date --> Agent --> Status}
url_robots_summary = robots_util.compute_url_date_agent_status(
    data=joined_robots, 
    # relevant_agents=agents_to_track)
    relevant_agents=[v for vs in agent_groups_to_track.values() for v in vs])

Num robot URLs loaded: 830
Earliest time: 2016-01-01
Last time: 2024-04-19
Num robot URLs loaded: 6230
Earliest time: 2016-01-01
Last time: 2024-04-19


In [6]:
# URL --> Date --> ToS-suburl --> {"verdict": X, "evidence": Y}
tos_policies = io.read_json(FPATH_TO_TOS_DATA)
print(f"Num ToS URLs: {len(tos_policies)}")

Num ToS URLs: 3068


In [39]:
url_to_info = analysis_util.extract_url_annotations(DIRPATHS_TO_ANNOTATED_TASKS)
url_results_df = analysis_util.process_url_annotations(url_to_info)
url_results_df = analysis_util.encode_size_columns(url_results_df, url_token_lookup)
url_results_df = robots_util.encode_latest_tos_robots_into_df(
    url_results_df, tos_policies, url_robots_summary,
    COMPANIES_TO_ANALYZE
)

6663 rows before filtering.
3017 rows after filtering. 1550 issues, 2096 unannotated.
7059


In [56]:
c4_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_c4_urls, 
    "c4", 
    "pretrain_data/corpus_token_bucket_counts/c4_buckets.csv",
    url_token_lookup,
    verbose=False
)
rf_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_rf_urls, 
    "rf", 
    "pretrain_data/corpus_token_bucket_counts/rf_buckets.csv",
    url_token_lookup,
    verbose=False
)
dolma_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_dolma_urls, 
    "dolma", 
    "pretrain_data/corpus_token_bucket_counts/dolma_buckets.csv",
    url_token_lookup,
    verbose=False
)

Head sample size: 1241
Rand sample size: 1056
Head sample size: 1155
Rand sample size: 1056
Head sample size: 1013
Rand sample size: 1056


In [58]:
results_df = analysis_util.analyze_url_variable_correlations(
    url_results_df, [100, 500, 1000], 
    c4_estimates,
    rf_estimates,
    dolma_estimates,
)

# Convert the dataframe to a LaTeX table
latex_table = results_df.to_latex(index=True, escape=True, float_format="{:.1f}".format)
print(latex_table)

Num URLs in random sample: 1056
\begin{tabular}{lrrrrlll}
\toprule
 & Top 100 & Top 500 & Top 1000 & Random & C4 & RW & Dolma \\
\midrule
User Content & 20.4 & 16.2 & 17.1 & 19.1 & 31.1 & 58.0 & 28.7 \\
Paywall & 36.4 & 38.6 & 35.9 & 1.8 & 3.5 & 4.0 & 2.5 \\
Ads & 58.0 & 60.6 & 55.4 & 5.8 & 34.2 & 60.2 & 24.9 \\
Modality: Image & 98.9 & 99.2 & 99.6 & 94.0 & 101.5 & 124.0 & 113.3 \\
Modality: Video & 96.6 & 96.1 & 96.3 & 19.9 & 34.4 & 34.5 & 25.2 \\
Modality: Audio & 93.2 & 95.3 & 95.2 & 3.2 & 33.0 & 12.0 & 12.8 \\
Sensitive Content & 0.0 & 0.3 & 0.4 & 0.6 & 1.6 & 0.2 & 5.2 \\
services\_Academic & 8.0 & 8.0 & 10.5 & 4.8 & 3.1 & 2.8 & 1.8 \\
services\_Blogs & 3.4 & 4.4 & 3.6 & 16.5 & 29.5 & 25.3 & 17.6 \\
services\_E-Commerce & 9.1 & 11.6 & 10.6 & 12.9 & 34.1 & 49.9 & 18.8 \\
services\_Encyclopedia/Database & 17.1 & 11.8 & 10.6 & 0.8 & 10.4 & 4.3 & 48.8 \\
services\_Government & 2.3 & 3.3 & 2.8 & 0.8 & 0.8 & 0.9 & 0.6 \\
services\_News/Periodicals & 48.9 & 52.6 & 50.5 & 6.8 & 14.8 & 22.0

In [ ]:
# clean up and review tokens vs URLs.

In [147]:
def prepare_altair_stacked_services_paywall(
    df,
    x_value='instance_count', 
    token_key='c4 tokens',
    width=800, 
    height=400,
):
    # Create a new column for the paywall/ads combination with custom labels
    df['paywall_ads'] = df.apply(lambda row: 'Base' if not row['Paywall'] and not row['Ads'] else
                                             'Ads' if not row['Paywall'] and row['Ads'] else
                                             'Paywall' if row['Paywall'] and not row['Ads'] else
                                             'Both Paywall & Ads', axis=1)

    # Explode the 'services' column to create a row for each service
    df = df.explode('Services')

    # Calculate the instance rate and total tokens
    instance_counts = df.groupby(['Services', 'paywall_ads']).size().reset_index(name='instance_count')
    total_tokens = df.groupby(['Services', 'paywall_ads'])[token_key].sum().reset_index(name='total_tokens')

    # Merge the instance counts and total tokens
    merged_df = instance_counts.merge(total_tokens, on=['Services', 'paywall_ads'])

    merged_df['percent'] = merged_df.groupby(['Services', 'paywall_ads'])[x_value].transform('sum') / merged_df.groupby('Services')[x_value].transform('sum') * 100
    services_ordered = merged_df.groupby('Services')[x_value].sum().sort_values(ascending=False).index.tolist()

    paywall_ads_order = ['Base', 'Ads', 'Paywall', 'Both Paywall & Ads']
    paywall_ads_colors = ['#8dd3c7', '#fb8072', '#ffffb3', '#bebada']

    return plot_altair_stacked_services_paywall(
        merged_df, x_axis=x_value, y_axis="Services", stack_axis="paywall_ads", stack_order=paywall_ads_order, stack_colors=paywall_ads_colors, width=width, height=height)

def plot_altair_stacked_services_paywall(
    merged_df, 
    x_axis, 
    y_axis,
    stack_axis,
    stack_order,
    stack_colors,
    width=400, 
    height=400,
):
    """
    Plots a horizontal stacked bar chart with service categories on the y-axis
    and the specified x-value (instance rate or total tokens) on the x-axis.
    The bars are stacked based on the paywall and ads values.

    Args:
        df (pandas.DataFrame): The input dataframe with columns 'url', 'tokens',
            'services', 'paywall', and 'ads'.
        x_value (str, optional): The value to use for the x-axis. Can be 'instance_rate'
            or 'total_tokens'. Defaults to 'instance_rate'.
        width (int, optional): The width of the chart. Defaults to 800.
        height (int, optional): The height of the chart. Defaults to 400.

    Returns:
        altair.Chart: The stacked bar chart.
    """
    # Create a categorical data type with the desired ordering
    merged_df[stack_axis] = pd.Categorical(
        merged_df[stack_axis],
        categories=stack_order,
        ordered=True
    )
    merged_df = merged_df.sort_values(by=stack_axis)
    
    # Create the chart
    chart = alt.Chart(merged_df).mark_bar().encode(
        x=alt.X(f"{x_axis}:Q", title=x_axis.replace('_', ' ').title()),
        y=alt.Y(
            f'{y_axis}:N', 
            sort='-x',
            # sort=services_ordered,
            title='Service Categories'
        ),
        color=alt.Color(f'{stack_axis}:N', title=None,
                        sort=stack_order,
                        scale=alt.Scale(
                            domain=stack_order,
                            range=stack_colors,
                        ),
                        legend=alt.Legend(
                            orient='top',
                            title=None,
                            labelFontSize=12,
                            labelPadding=5,
                            labelColor='black',
                            labelFontWeight='bold'
                        )
                        ),
        order="order:Q",
        # tooltip=['Services', 'paywall_ads', f"{x_value}:Q"]
    ).properties(
        width=width,
        height=height
    )

    return chart

In [148]:
chart = prepare_altair_stacked_services_paywall(df, x_value='total_tokens')
chart.show()

alt.Chart(...)

In [219]:
def prepare_altair_wildchat(fpath):
    df = pd.read_csv(fpath)
    category_map = {
        'Coding Composition': [
            'Coding composition (fixing, debugging, or help)',
            'Coding composition',
        ],
        'Explanation and Reasoning': [
            'Asking for an explanation, reasoning, or help solving a puzzle, or math problem',
            'Asking for an explanation, reasoning, or help solving a puzzle or math problem',
        ],
        'Creative Composition': [
            'Creative composition',
            'Creative composition (such as role-playing, fictional story writing or continuation)'
        ],
        'Academic Composition': [
            'Academic composition (such as non-fiction essay writing, continuation, or fixing)',
            'Academic composition'
        ],
        'General Information': ['General informational requests'],
        'Sexual/Illegal Content': ['Sexual or illegal content requests'],
        'Translation': ['Translation'],
        'Brainstorming and Planning': ['Brainstorming, planning, or ideation'],
        'Self-help & welf-harm': [
            'Self-help, advice seeking, or self-harm',
            'Self-help, advice seeking'
        ],
        'Organization Information': [
            'Information requests specifically about organizations, companies, or persons'
        ],
        'E-commerce Information': [
            'E-commerce or information requests about products and purchasing'
        ],
        'News': [
            'News or recent events informational requests',
        ],
        'Other': ['Other', np.nan],
    }
    RELEVANT_CATEGORIES = {
        "News": "Relevance to Websites",
        "E-commerce Information": "Relevance to Websites",
        "Organization Information": "Relevance to Websites",
        "Academic Composition": "Relevance to Websites",
        "Sexual/Illegal Content": "NSFW Content",
    }
    
    inverse_mapper = {v: k for k, vs in category_map.items() for v in vs}

    def safe_literal_eval(val):
        if "[" not in val:
            return [val]
        else:
            try:
                return ast.literal_eval(val)
            except (ValueError, SyntaxError):
                print(val)
                return None  # or some other default value or handling mechanism
    
    df['Types of Service'] = df['Types of Service'].apply(safe_literal_eval)

    # df['Types of Service'] = df['Types of Service'].apply(ast.literal_eval)
    df = wildchat_df.explode('Types of Service')
    print(len(df))

    cat_df = []
    for category, count in Counter(df['Types of Service'].tolist()).most_common():
        cat_df.append({
            "category": inverse_mapper[category],
            "count": count,
            "stack": RELEVANT_CATEGORIES.get(inverse_mapper[category], "Less Relevant to Websites"),
        })
        
    return plot_altair_stacked_services_paywall(
        pd.DataFrame(cat_df), 
        x_axis="count", 
        y_axis="category", 
        stack_axis="stack", 
        stack_order=["Relevance to Websites", "Less Relevant to Websites", "NSFW Content"], 
        stack_colors=['#fb8072', '#8dd3c7', '#ffffb3'] #, '#bebada'], 
    )


In [220]:
prepare_altair_wildchat("test_data/wildchat_annotations.csv")

1323


alt.Chart(...)

In [ ]:
# Calculate underlying percents. Look at random/head, dolma/rf/c4.

In [59]:
# df_uc = df[df["sample"] != "random"]

In [60]:
# # Normalize by token count
# # ax = plot_modality_stacked_bar(df, normalize_by='token_count')

# # Normalize by URL count
# ax = plot_modality_stacked_bar(df_uc, normalize_by='url_count')

In [452]:
import pandas as pd

# Explode the 'domains' column
dfx = df.explode('Domains')

# Get the unique domain categories
domain_categories = dfx['Domains'].unique()

# Create a new column for each domain category with True/False values
for category in domain_categories:
    df[f'domain_{category}'] = df['Domains'].apply(lambda x: category in x)

# Filter the DataFrame to only include rows with video modality
df_with_videos = df[df['Modality: Video']]

# Calculate the portion of URLs with videos for each domain category
portions_by_category = {}
for category in domain_categories:
    category_col = f'domain_{category}'
    total_videos = len(df_with_videos)
    videos_in_category = df_with_videos[category_col].sum()
    portion = videos_in_category / total_videos
    portions_by_category[category] = portion

# Print the results
tot_portions = sum(portions_by_category.values())
for category, portion in portions_by_category.items():
    print(f"{category}: {100 * portion / tot_portions :.2f}%")
# print(sum(tot_portions))

Entertainment & Culture: 20.21%
News: 29.28%
Legal & Policy: 4.30%
Science, Academia, & Technology: 11.14%
General Information & Education: 11.22%
Books: 1.94%
Business & E-Commerce: 11.65%
Social Media/Forums: 4.18%
Other: 3.08%
Blogs: 3.00%
nan: 0.00%
